# NFL Historic to Present Matches

In [2]:
import pandas as pd
import time
import re

## Data Collection and Parsing

### Prepare table of past scores to use

In [64]:
all_games = pd.read_html("https://www.pro-football-reference.com/boxscores/game-scores.htm")

In [66]:
all_games_df = pd.DataFrame(all_games[0])

In [72]:
print(f"Total number of unique score combinations: {all_games_df.shape[0]}")

Total number of unique score combinations: 1069


In [67]:
WinScores = all_games_df["PtsW"].tolist()
LossScores = all_games_df["PtsL"].tolist()

In [69]:
print(f"Sample Win Score: {WinScores[:5]}")
print(f"Sample Loss Score: {LossScores[:5]}")


Sample Win Score: [20, 27, 17, 23, 24]
Sample Loss Score: [17, 24, 14, 20, 17]


### Obtain all games for each scoreline

In [ ]:
cols = ['Rk', 'Week', 'Day', 'Date', 'Unnamed: 4', 'Winner/tie', 'Unnamed: 6', 'Loser/tie', 'Unnamed: 8',
        'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL']
out_df = pd.DataFrame(columns=cols)
for W, L in zip(WinScores, LossScores):
    temp_df = pd.read_html(f"https://www.pro-football-reference.com/boxscores/game_scores_find.cgi?pts_win={W}&pts_lose={L}")[0]
    out_df = out_df.append(temp_df)
    time.sleep(1)

In [68]:
out_df = out_df.rename(columns={"Unnamed: 6": "AtHome"})
out_df.to_csv("historic_match_scores.csv")

In [9]:
out_df = pd.read_csv("historic_match_scores.csv")
out_df["Date"] = out_df["Date"].astype("datetime64") 
out_df["Week"] = [row.lstrip().rstrip() for row in out_df["Week"].astype("str")]

In [50]:
# PtsW - Points scored by the winning team
# PtsL - Points score by the losing team
# YdsW - Yards gained by the winning team
# YdsL - Yards gained by the losing team
# TOW - Turnovers by winning Team
# TOL - Turnovers by losing Team

In [4]:
def get_home_team(Winner, Loser, AtHome):
    if AtHome == "@":
        HomeTeam = Loser
    else:
        HomeTeam = Winner
    
    return HomeTeam

In [10]:
out_df["HomeTeam"] = out_df.apply(lambda row : get_home_team(row["Winner/tie"],
                                  row["Loser/tie"], row["AtHome"]), axis = 1)

In [6]:
def get_season(DateField):
    """This function parses the given match date to process and the return the year of the season being played.

    Args:
        DateField (datetime): The column/field that contains the date that the match was played upon

    Returns:
        Int: The year of the season being played
    """
    if (DateField.month >= 1) & (DateField.month <= 3):
        SeasonYear = DateField.year -1
    else:
        SeasonYear = DateField.year
    return SeasonYear

In [58]:
out_df = out_df.drop(["Unnamed: 4", "Unnamed: 8"], axis=1, inplace=True)

In [11]:
out_df["Season"] = out_df.apply(lambda row : get_season(row["Date"]), axis = 1)

### Wild Card
* Added in 1978 (March), being the 1977 season

## 2020 Season

In [8]:
df_2020 = out_df[out_df["Season"] == 2020]
df_2020["Week"] = [row.lstrip().rstrip() for row in df_2020["Week"].astype("str")]

/Users/alessiomorteo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [98]:
print(f"Number of games played: {len(df_2020)}")

Number of games played: 269


In [62]:
wk_occurence_2020 = df_2020.groupby(["Week"]).size().reset_index().rename(columns={0:'count'}).sort_values(by="count", ascending=False)

### Week Types

In [12]:
unique_weeks = {}
for season in out_df["Season"].unique():
    unique_weeks[season] = {}
    temp_df = out_df[out_df["Season"] == season]
    weeks_found = temp_df["Week"].unique().tolist()
    unique_weeks[season] = [str(i) for i in weeks_found]

In [13]:
all_time_unique = sorted({x for v in unique_weeks.values() for x in v})

In [14]:
out_df["Week"] = out_df["Week"].replace("Conf. Champ", "Champ")

### Max Month

In [15]:
max_week = {}
for season in out_df["Season"].unique():
    max_week[season] = {}
    temp_df = out_df["Week"][out_df["Season"] == season].astype("str")
    week_values = [int(i) for i in temp_df if not re.search('[a-zA-Z]', i)]
    #week_values = [int(i) for i in week_values]
    max_week[season] = max(week_values)

### Replacement Keys

In [22]:
week_correction_dict = {}
for season in max_week.keys():
    week_correction_dict[season] = {}
    temp_unique_weeks = unique_weeks[season]
    temp_max_weeks = max_week[season]
    
    if (season < 1966):
        week_correction_dict[season]["Division"] =temp_max_weeks + 1
        week_correction_dict[season]["Champ"] = temp_max_weeks + 2
    elif (season >= 1966) & (season <= 1978):
        week_correction_dict[season]["Division"] =temp_max_weeks + 1
        week_correction_dict[season]["Champ"] = temp_max_weeks + 2
        week_correction_dict[season]["Super Bowl"] = temp_max_weeks + 3
    else:
        week_correction_dict[season]["Wild Card"] = temp_max_weeks + 1
        week_correction_dict[season]["Division"] =temp_max_weeks + 2
        week_correction_dict[season]["Champ"] = temp_max_weeks + 3
        week_correction_dict[season]["Super Bowl"] = temp_max_weeks + 4

In [47]:
def do_complete_week_col(season_value, week_value):
    for old, new in week_correction_dict[season_value].items():
        new_week = week_value.replace(old, str(new))
    return new_week

In [48]:
out_df["ModifiedWeek"] = out_df.apply(lambda row : do_complete_week_col(row["Season"], row["Week"]), axis=1)

In [60]:
out_df

## Super Bowl History

In [61]:
sb_games = out_df[out_df["Week"] == "Super Bowl"]

TypeError: 'NoneType' object is not subscriptable